In [1]:
%load_ext autoreload
%autoreload 2

### Search for a geographic reference product for the Stack 

In [2]:
import asf_search as asf

results = asf.product_search('S1A_IW_SLC__1SDV_20220215T225119_20220215T225146_041930_04FE2E_9252-SLC')
# results = asf.product_search('S1_181296_IW1_20220219T125501_VV_10AF-BURST')
reference = results[0]
reference

### Create an `asf_search.Stack` object from the reference scene `asf_search.Products.S1Product.S1Product` object

Stack accepts ASFSearchOptions, which you can use to limit its temporal range.

This creates a fully connected stack based on the results for a given `asf.Stack` search.

In [3]:

args = asf.ASFSearchOptions(
    **{"start": '2022-01-01', "end": '2023-03-02'}
)

args = asf.ASFSearchOptions(
    **{"start": '2022-01-01', "end": '2022-04-02'}
)

stack = asf.Stack(reference, opts=args)
stack.full_stack




[Pair(S1A_IW_SLC__1SDV_20220110T225120_20220110T225147_041405_04EC56_3D53, S1A_IW_SLC__1SDV_20220122T225120_20220122T225147_041580_04F215_AD4C),
 Pair(S1A_IW_SLC__1SDV_20220110T225120_20220110T225147_041405_04EC56_3D53, S1A_IW_SLC__1SDV_20220203T225119_20220203T225146_041755_04F804_032E),
 Pair(S1A_IW_SLC__1SDV_20220110T225120_20220110T225147_041405_04EC56_3D53, S1A_IW_SLC__1SDV_20220215T225119_20220215T225146_041930_04FE2E_9252),
 Pair(S1A_IW_SLC__1SDV_20220110T225120_20220110T225147_041405_04EC56_3D53, S1A_IW_SLC__1SDV_20220227T225119_20220227T225146_042105_050431_987E),
 Pair(S1A_IW_SLC__1SDV_20220110T225120_20220110T225147_041405_04EC56_3D53, S1A_IW_SLC__1SDV_20220311T225119_20220311T225146_042280_050A1F_B99E),
 Pair(S1A_IW_SLC__1SDV_20220110T225120_20220110T225147_041405_04EC56_3D53, S1A_IW_SLC__1SDV_20220323T225119_20220323T225146_042455_051017_6160),
 Pair(S1A_IW_SLC__1SDV_20220122T225120_20220122T225147_041580_04F215_AD4C, S1A_IW_SLC__1SDV_20220203T225119_20220203T225146_041755

### At this point, `stack.full_stack` == `stack.subset_stack` because we haven't yet removed any pairs

Also note that `stack.remove_list` is empty, and the `Stack` is fully-connected, which can be seen by the fact that `stack.connected_substacks` contains only a single item.

In [4]:
print(f'len(stack.full_stack): {len(stack.full_stack)}')
print(f'len(stack.subset_stack): {len(stack.subset_stack)}')
print(f'len(stack.remove_list): {len(stack.remove_list)}')
print(f'len(stack.connected_substacks): {len(stack.connected_substacks)}')
print(f'stack.subset_stack == stack.full_stack: {stack.subset_stack == stack.full_stack}')

len(stack.full_stack): 21
len(stack.subset_stack): 21
len(stack.remove_list): 0
len(stack.connected_substacks): 1
stack.subset_stack == stack.full_stack: True


### Remove some Pairs from the stack

The `Pair`s have been added to `stack.remove_list` and removed from `stack.subset_stack`

In [5]:
stack.remove_pairs(stack.full_stack[1:11])

print(f'len(stack.full_stack): {len(stack.full_stack)}')
print(f'len(stack.subset_stack): {len(stack.subset_stack)}')
print(f'len(stack.remove_list): {len(stack.remove_list)}')
print(f'stack.subset_stack == stack.full_stack: {stack.subset_stack == stack.full_stack}')

len(stack.full_stack): 21
len(stack.subset_stack): 11
len(stack.remove_list): 10
stack.subset_stack == stack.full_stack: False


### Try removing a `Pair` that is not part of `stack.full_stack`

This will raise a warning indicating that the Pair cannot be removed since it is not part of the Stack

In [6]:
# Build a Pair from scenes that are not included in the existing Stack
results_1 = asf.product_search('S1A_IW_SLC__1SDV_20250903T225120_20250903T225147_060830_0792D3_868A-SLC')
results_2 = asf.product_search('S1A_IW_SLC__1SDV_20250822T225120_20250822T225147_060655_078BEA_2065-SLC')
my_pair = asf.Pair(results_1[0], results_2[0])

stack.remove_pairs([my_pair])

/Users/aflewandowski/Documents/asf_search/Discovery-asf_search/asf_search/Stack.py:120: PairNotInFullStackWarning: warning: Pair(S1A_IW_SLC__1SDV_20250903T225120_20250903T225147_060830_0792D3_868A, S1A_IW_SLC__1SDV_20250822T225120_20250822T225147_060655_078BEA_2065) is not in full_stack
  warnings.warn(PairNotInFullStackWarning(msg))


### Examine `stack.connected_substacks`

The set of previously removed `Pair`s was intentionally selected to break the `stack.subset_stack` into two fully-connected `Stack` networks for illustrative purposes.

We can view these connected substacks by examing `stack.connected_substacks`, which contains a list of all connected substacks within `stack.subset_stack`

In [7]:
print(f'len(stack.connected_substacks): {len(stack.connected_substacks)}')

for s in stack.connected_substacks:
    print(s)

len(stack.connected_substacks): 2
{Pair(S1A_IW_SLC__1SDV_20220110T225120_20220110T225147_041405_04EC56_3D53, S1A_IW_SLC__1SDV_20220122T225120_20220122T225147_041580_04F215_AD4C): Pair(S1A_IW_SLC__1SDV_20220110T225120_20220110T225147_041405_04EC56_3D53, S1A_IW_SLC__1SDV_20220122T225120_20220122T225147_041580_04F215_AD4C)}
{Pair(S1A_IW_SLC__1SDV_20220203T225119_20220203T225146_041755_04F804_032E, S1A_IW_SLC__1SDV_20220215T225119_20220215T225146_041930_04FE2E_9252): Pair(S1A_IW_SLC__1SDV_20220203T225119_20220203T225146_041755_04F804_032E, S1A_IW_SLC__1SDV_20220215T225119_20220215T225146_041930_04FE2E_9252), Pair(S1A_IW_SLC__1SDV_20220203T225119_20220203T225146_041755_04F804_032E, S1A_IW_SLC__1SDV_20220227T225119_20220227T225146_042105_050431_987E): Pair(S1A_IW_SLC__1SDV_20220203T225119_20220203T225146_041755_04F804_032E, S1A_IW_SLC__1SDV_20220227T225119_20220227T225146_042105_050431_987E), Pair(S1A_IW_SLC__1SDV_20220203T225119_20220203T225146_041755_04F804_032E, S1A_IW_SLC__1SDV_20220311T

### Add one of the removed `Pair`s back to `stack.subset_stack`

Adding this `Pair` back once again fully-connects `stack.subset_stack`

In [9]:
stack.add_pairs([stack.full_stack[1]])
print(f'len(stack.connected_substacks): {len(stack.connected_substacks)}')

len(stack.connected_substacks): 1


### You can also add `Pairs` that are not part of the original `stack.full_stack`

This is useful if you need to add custom `Pairs` that were not included during automated `Stack.full_stack` creation. We created such a `Pair` earlier in the notebook (my_pair).

Note that adding this `Pair` once again splits `stack.subset_stack` into 2 connected substacks.

In [ ]:
stack.add_pairs([my_pair])
print(f'len(stack.connected_substacks): {len(stack.connected_substacks)}')

len(stack.connected_substacks): 2
